In [1]:
import altair as alt
import pandas as pd
import numpy as np
import warnings
#warnings.simplefilter(action='ignore', category="SettingWithCopyWarning")
 

In [7]:
df = pd.read_csv("../data/chicago_bears_coaching_history.csv")
df = df.query("Year >= 1950")

df = df[~df.Coach.eq("Matt Eberflus")]
df = df.reset_index()
df["Made Playoffs"] = ~df.Result.isnull()
df_agg = df.groupby("Coach", as_index=False).agg({"W":"sum", "L":"sum", "W-L%":"mean"}).round(2)

In [8]:
df

,index,Year,Coach,G,W,L,T,W-L%,G.1,W.1,L.1,Result,Offense,Defense,Made Playoffs
0,1,2021.0,Matt Nagy,17,6.0,11.0,0.0,0.353,NaN,NaN,NaN,NaN,Bill Lazor,Sean Desai,False
1,2,2020.0,Matt Nagy,16,8.0,8.0,0.0,0.500,1.0,0.0,1.0,Lost WC,Bill Lazor,Chuck Pagano,True
2,3,2019.0,Matt Nagy,16,8.0,8.0,0.0,0.500,NaN,NaN,NaN,NaN,Mark Helfrich,Chuck Pagano,False
3,4,2018.0,Matt Nagy,16,12.0,4.0,0.0,0.750,1.0,0.0,1.0,Lost WC,Mark Helfrich,Vic Fangio,True
4,5,2017.0,John Fox,16,5.0,11.0,0.0,0.313,NaN,NaN,NaN,NaN,Dowell Loggains,Vic Fangio,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,68,1954.0,George Halas,12,8.0,4.0,0.0,0.667,0.0,0.0,0.0,NaN,NaN,NaN,False
68,69,1953.0,George Halas,12,3.0,8.0,1.0,0.273,0.0,0.0,0.0,NaN,NaN,NaN,False
69,70,1952.0,George Halas,12,5.0,7.0,0.0,0.417,0.0,0.0,0.0,NaN,NaN,NaN,False
70,71,1951.0,George Halas,12,7.0,5.0,0.0,0.583,0.0,0.0,0.0,NaN,NaN,NaN,False


In [3]:
# have a bar char where bar has two an x start and end date and sort by 
#chart_title = ""
selector = alt.selection_multi(fields=['Coach'])


y_cfgs = {"shorthand":"Coach", "title": ""}
x_cfgs = {"shorthand":"Year:O", "title":"Year"}
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"


In [4]:
heat = alt.Chart(df, width=1200, height=800, title="Chicago Bears Coaching History").mark_rect()\
.encode(
    y=alt.Y(**y_cfgs, sort=alt.EncodingSortField(field='Year', order='descending')),
    x=alt.X(**x_cfgs),
     color=alt.Color("Made Playoffs", scale=alt.Scale(domain=[True, False], range=[secondary_color, "white"])),
    tooltip=[
        alt.Tooltip("Coach", title="Coach"), 
        "W-L%", 
        alt.Tooltip("Year", title="Year"), 
        alt.Tooltip("W", title="Total Wins"), 
        alt.Tooltip("L", title="Losses"), 
        alt.Tooltip("Result", title="Result"),
        alt.Tooltip("Offense", title="Off. Coord."),
        alt.Tooltip("Defense", title="Def. Coord."),

        ]
).transform_filter(
    selector
)

In [5]:
chart_title2 = "Career Winning %"
y2_cfgs = {"shorthand":"Coach", "title": "Coach"}
x2_cfgs = {"shorthand":"W-L%", "title":"W-L%"}

bars = alt.Chart(df_agg, width=400, height=800, title="Career Winning%")\
.mark_bar(size=30, opacity=1, color="white")\
.encode(
    y=alt.Y(**y2_cfgs, sort=alt.EncodingSortField(field="W-L%", order="descending")),
    x=alt.X(**x2_cfgs),
    tooltip=[alt.Tooltip("Coach", title="Coach"),alt.Tooltip("W", title="Total Wins"), alt.Tooltip("L", title="Total Losses"), alt.Tooltip("W-L%", title="W-L%")]
).add_selection(selector).resolve_scale(y="independent")

In [6]:
(bars | heat).configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=17, titleFontSize=20)\
.configure_axisBottom(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=15, titleFontSize=17)\
.configure_legend(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10,cornerRadius=10, labelFontSize=17, titleFontSize=17)\
.configure_title(color=secondary_color, fontSize=17)

alt.HConcatChart(...)